In [7]:
from helpers import stats, weapon_types, prof_translator, item_translator, proffesions
from pprint import pprint
import pandas as pd


armor_dfs = {} 

for key, val in proffesions.items():
    armor_dfs[key] = pd.read_csv(f'items/{val}_Zbroje.csv')

pprint(armor_dfs)

{'b':      lvl    prof type                              name  \
0     30      bh   ar  Skórzana kurta jeźdźcy błyskawic   
1     30       b   ar                  Ubłocona narzuta   
2     30  wpbhtm   ar               Szata żony grabarza   
3     30      bh   ar  Skórzana kurta jeźdźca błyskawic   
4     30       b   ar               Nawiedzone kimono I   
..   ...     ...  ...                               ...   
280  300  wpbhtm   ar         Zbroja obrońcy smoków XVI   
281  300       b   ar   Imperialna zbroja mistrzów XIII   
282  300       b   ar          Napierśnik władcy wiatru   
283  300       b   ar     Napierśnik wirujących płatków   
284  300       b   ar       Napierśnik lodowej kostuchy   

                                                 stats  
0    {'armor': '95', 'poison_resist': '15', 'all_fe...  
1    {'armor': '107', 'poison_resist': '8', 'crit_s...  
2    {'armor': '107', 'crit_strike': '2', 'all_feat...  
3    {'armor': '101', 'poison_resist': '12', 'all_f...  
